In [28]:
#Importing the necesary packages
import os
from os import path
import shutil
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import datasets, transforms, models
from torch.autograd import Variable
import numpy as np
from matplotlib import pyplot as plt

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
PATH = '/content/drive/MyDrive/Soft Computing Lab/Dataset_Assignment01/'
os.listdir(PATH)

['firmware', 'imagery', 'allimages']

In [31]:
def showRawTrainingSamples(csv_filename):
  df = pd.read_csv(PATH + csv_filename)
  print(csv_filename)
  print(df.columns)
  return df

In [32]:
firmware_csv = showRawTrainingSamples('firmware/firmware.csv')

firmware/firmware.csv
Index(['filename', 'prefilename', 'extension', 'class', 'target', 'pixel1',
       'pixel2', 'pixel3', 'pixel4', 'pixel5',
       ...
       'pixel1015', 'pixel1016', 'pixel1017', 'pixel1018', 'pixel1019',
       'pixel1020', 'pixel1021', 'pixel1022', 'pixel1023', 'pixel1024'],
      dtype='object', length=1029)


In [33]:
#here i make a function for taking filename and class from csv file.
def dropColumns(csv_file):
  csv_file = csv_file[['filename', 'class']]
  print(csv_file)
  print(csv_file.iloc[:5, :])   
  return csv_file

In [34]:
firmware_csv = dropColumns(firmware_csv)

                                                filename       class
0      x86__64__lsb__unix-system-v__clang-3.8.0__O0__...     malware
1      x86__64__lsb__unix-system-v__clang-3.8.0__O0__...     malware
2      x86__64__lsb__unix-system-v__clang-3.8.0__O0__...     malware
3      x86__64__lsb__unix-system-v__clang-3.8.0__O0__...     malware
4      x86__64__lsb__unix-system-v__clang-3.8.0__O0__...     malware
...                                                  ...         ...
38882  x86__64__lsb__unix-system-v__llvm-obfuscator-3...  benignware
38883  x86__64__lsb__unix-system-v__llvm-obfuscator-3...  benignware
38884  x86__64__lsb__unix-system-v__llvm-obfuscator-3...  benignware
38885  x86__64__lsb__unix-system-v__clang-3.9.0__O0__...  benignware
38886  x86__64__lsb__unix-system-v__llvm-obfuscator-4...  benignware

[38887 rows x 2 columns]
                                            filename    class
0  x86__64__lsb__unix-system-v__clang-3.8.0__O0__...  malware
1  x86__64__lsb__unix-

In [35]:
print(len(firmware_csv))

38887


In [ ]:
# dataset_A = url + 'Dataset A.zip'
# with ZipFile(dataset_A, 'r') as zip:
#   zip.extractall()

In [9]:
data_labels = pd.read_csv(PATH + 'firmware/firmware.csv', usecols = ['filename', 'class'])
print(data_labels.shape)
data_labels.head()

(38887, 2)


,filename,class
0,x86__64__lsb__unix-system-v__clang-3.8.0__O0__...,malware
1,x86__64__lsb__unix-system-v__clang-3.8.0__O0__...,malware
2,x86__64__lsb__unix-system-v__clang-3.8.0__O0__...,malware
3,x86__64__lsb__unix-system-v__clang-3.8.0__O0__...,malware
4,x86__64__lsb__unix-system-v__clang-3.8.0__O0__...,malware


In [37]:
TRAIN_PATH = PATH + 'allimages'
#os.mkdir(TRAIN_PATH)

def processImages(folder_name):
  src = PATH + 'imagery/' + folder_name + '/'
  dir_folders = os.listdir(src)
  for dir_name in dir_folders:
    file_name = os.path.join(src, dir_name)
    if os.path.isfile(file_name):
      shutil.copy(file_name, TRAIN_PATH) 

In [ ]:
processImages('benignware')
print('benignware Done')
processImages('gray')
print('gray Done')
processImages('hackware')
print('hackware Done')
processImages('malware')
print('malware Done')

benignware Done
gray Done
hackware Done
malware Done


In [ ]:
processImages('benignware')
print('benignware Done')

In [ ]:
processImages('gray')
print('gray Done')

gray Done


In [ ]:
processImages('hackware')
print('hackware Done')

hackware Done


In [ ]:
processImages('malware')
print('malware Done')

malware Done


In [19]:
TRAIN_PATH = PATH + 'allimages'

In [40]:
class Dataset(Dataset):
    def __init__(self, df, root, transform=None):
        self.data = df
        self.root = root
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        item = self.data.iloc[index]
        
        path = self.root + "/" + item[0]
        image = Image.open(path).convert('L')
        label = item[1]
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label

In [41]:
mean = [0.5,]
std = [0.5, ]

train_transform = transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

test_transform = transforms.Compose([
        transforms.Resize(28),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
])

train_data  = Dataset(data_labels, TRAIN_PATH, train_transform)
test_data = Dataset(data_labels, TRAIN_PATH, test_transform)

print("Training Samples: ", len(train_data))

Trainig Samples:  38887


# **Base Parameters**

In [42]:
# Hyperparameters

batch_size = 20
num_iters = 2000
input_dim = 28*28 # num_features = 784
num_hidden = 100 # num of hidden nodes
output_dim = 10

learning_rate = 0.001  # More power so we can learn faster! previously it was 0.001

num_epochs = num_iters / (len(train_data) / batch_size)
num_epochs = int(num_epochs)
print(num_epochs)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

1


In [43]:
# split data 20% for testing
test_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data)

# mix data
# index of num of train
indices = list(range(num_train))

# random the index
np.random.shuffle(indices)
split = int(np.floor(test_size * num_train))

# divied into two part
train_idx, test_idx = indices[split:], indices[:split]

# define the sampler
train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)

# prepare loaders
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=batch_size,
    sampler=train_sampler)

test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=batch_size,
    sampler=test_sampler)

print("Train dataloader:{}".format(len(train_loader)))
print("Test dataloader:{}".format(len(test_loader)))

Train dataloader:1556
Test dataloader:389


In [44]:
class DeepNeuralNetworkModel(nn.Module):
    def __init__(self, input_size, num_classes, num_hidden):
        super().__init__()
       
        self.linear_1 = nn.Linear(input_size, num_hidden)
        self.relu_1 = nn.ReLU()
#         self.softmax_1 = nn.Softmax(dim=1)
 
        self.linear_2 = nn.Linear(num_hidden, num_hidden)
        self.relu_2 = nn.ReLU()
#         self.softmax_2 = nn.Softmax(dim=1)
 
        self.linear_3 = nn.Linear(num_hidden, num_hidden)
        self.relu_3 = nn.ReLU()
#         self.softmax_3 = nn.Softmax(dim=1)
 
        self.linear_4 = nn.Linear(num_hidden, num_hidden)
        self.relu_4 = nn.ReLU()
#         self.softmax_4 = nn.Softmax(dim=1)
 
        self.linear_5= nn.Linear(num_hidden, num_hidden)
        self.relu_5= nn.ReLU()

 
        self.linear_out = nn.Linear(num_hidden, num_classes)
 
    def forward(self, x):
        out  = self.linear_1(x)
        out = self.relu_1(out)
        
        out  = self.linear_2(out)
        out = self.relu_2(out)
 
        out  = self.linear_3(out)
        out = self.relu_3(out)
 
        out  = self.linear_4(out)
        out = self.relu_4(out)
 
        out  = self.linear_5(out)
        out = self.relu_5(out)
        
        
        probas  = self.linear_out(out)
        return probas

In [45]:
model = DeepNeuralNetworkModel(input_size = input_dim, num_classes = output_dim, num_hidden = num_hidden)

model.to(device)

DeepNeuralNetworkModel(
  (linear_1): Linear(in_features=784, out_features=100, bias=True)
  (relu_1): ReLU()
  (linear_2): Linear(in_features=100, out_features=100, bias=True)
  (relu_2): ReLU()
  (linear_3): Linear(in_features=100, out_features=100, bias=True)
  (relu_3): ReLU()
  (linear_4): Linear(in_features=100, out_features=100, bias=True)
  (relu_4): ReLU()
  (linear_5): Linear(in_features=100, out_features=100, bias=True)
  (relu_5): ReLU()
  (linear_out): Linear(in_features=100, out_features=10, bias=True)
)

In [46]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [50]:
iteration_loss = []
iter = 0
for epoch in range(num_epochs):
    print('Epoch: ', epoch + 1)
    for i, (images, labels) in enumerate(train_loader):

        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images) 

        loss = criterion(outputs, labels)

        loss.backward()

        optimizer.step()

        iter += 1

        if iter % 10 == 0:        
            correct = 0
            total = 0
            for images, labels in test_loader:
               
                images = images.view(-1, 28*28).to(device)

                outputs = model(images)

                _, predicted = torch.max(outputs, 1)

                total += labels.size(0)

                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum() 
                else:
                    correct += (predicted == labels).sum()

            accuracy = 100 * correct.item() / total

            iteration_loss.append(loss.item())
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Epoch:  1


FileNotFoundError: ignored

In [ ]:
torch.save(model.state_dict(), 'model_assignment_1_1.pkl')

In [ ]:
# print (iteration_loss)
plt.plot(range(0,15550,10),it)
plt.ylabel('Loss')
plt.xlabel('Iteration')
plt.show()

In [ ]:
it = np.array(iteration_loss)
np.random.shuffle(it)

In [ ]:
print(accuracy)

In [ ]:
from sklearn.metrics import accuracy_score

def evaluate_model(test_dl, model):
    model.eval() 
    total = 0
    correct = 0
    for images, labels in test_dl:
        
#         image = images.reshape(28, 28, 1)
        print(images.shape)
    
        # Show result
               
        images = images.view(-1, 28*28).to(device)
        

        outputs = model(images)

        _, predicted = torch.max(outputs, 1)

        plt.imshow(images[0].reshape(28, 28, 1), cmap='gray')
        plt.title(f'Prediction: {predicted[0]} - Actual target: {labels[0]}')
        plt.show()


        total += labels.size(0)

        if torch.cuda.is_available():
            correct += (predicted.cpu() == labels.cpu()).sum() 
        else:
            correct += (predicted == labels).sum()

    acc = 100 * correct.item() / total
    return acc

In [ ]:
acc = evaluate_model(test_loader, model)
print('Accuracy: %.3f' % acc)

CNN

In [ ]:
class ConvNeuralNetworkModel(nn.Module):
    def __init__(self, input_size, num_classes, num_hidden):
        super().__init__()
       
        self.linear_1 = nn.Linear(input_size, num_hidden)
        self.relu_1 = nn.ReLU()
#         self.softmax_1 = cnn.Softmax(dim=1)
 
        self.linear_2 = nn.Linear(num_hidden, num_hidden)
        self.relu_2 = nn.ReLU()
#         self.softmax_2 = cnn.Softmax(dim=1)
 
        self.linear_3 = nn.Linear(num_hidden, num_hidden)
        self.relu_3 = nn.ReLU()
#         self.softmax_3 = cnn.Softmax(dim=1)
 
        self.linear_4 = nn.Linear(num_hidden, num_hidden)
        self.relu_4 = nn.ReLU()
#         self.softmax_4 = cnn.Softmax(dim=1)
 
        self.linear_5= nn.Linear(num_hidden, num_hidden)
        self.relu_5= nn.ReLU()

 
        self.linear_out = nn.Linear(num_hidden, num_classes)
 
    def forward(self, x):
        out  = self.linear_1(x)
        out = self.relu_1(out)
        
        out  = self.linear_2(out)
        out = self.relu_2(out)
 
        out  = self.linear_3(out)
        out = self.relu_3(out)
 
        out  = self.linear_4(out)
        out = self.relu_4(out)
 
        out  = self.linear_5(out)
        out = self.relu_5(out)
        
        
        probas  = self.linear_out(out)
        return probas

In [ ]:
model = ConvNeuralNetworkModel(input_size = input_dim, num_classes = output_dim, num_hidden = num_hidden)

model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
iteration_loss = []
iter = 0
for epoch in range(num_epochs):
    print('Epoch: ', epoch + 1)
    for i, (images, labels) in enumerate(train_loader):

        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images) 

        loss = criterion(outputs, labels)

        loss.backward()

        optimizer.step()

        iter += 1

        if iter % 10 == 0:        
            correct = 0
            total = 0
            for images, labels in test_loader:
               
                images = images.view(-1, 28*28).to(device)

                outputs = model(images)

                _, predicted = torch.max(outputs, 1)

                total += labels.size(0)

                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum() 
                else:
                    correct += (predicted == labels).sum()

            accuracy = 100 * correct.item() / total

            iteration_loss.append(loss.item())
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

In [ ]:
torch.save(model.state_dict(), 'model_assignment_1.pkl')

In [ ]:
# print (iteration_loss)
plt.plot(range(0,15550,10),it)
plt.ylabel('Loss')
plt.xlabel('Iteration')
plt.show()

In [ ]:
it = np.array(iteration_loss)
np.random.shuffle(it)

In [ ]:
print(accuracy)

In [ ]:
from sklearn.metrics import accuracy_score

def evaluate_model(test_dl, model):
    model.eval() 
    total = 0
    correct = 0
    for images, labels in test_dl:
        
#         image = images.reshape(28, 28, 1)
        print(images.shape)
    
        # Show result
               
        images = images.view(-1, 28*28).to(device)
        

        outputs = model(images)

        _, predicted = torch.max(outputs, 1)

        plt.imshow(images[0].reshape(28, 28, 1), cmap='gray')
        plt.title(f'Prediction: {predicted[0]} - Actual target: {labels[0]}')
        plt.show()


        total += labels.size(0)

        if torch.cuda.is_available():
            correct += (predicted.cpu() == labels.cpu()).sum() 
        else:
            correct += (predicted == labels).sum()

    acc = 100 * correct.item() / total
    return acc

In [ ]:
acc = evaluate_model(test_loader, model)
print('Accuracy: %.3f' % acc)